In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
# Загружаем модель resnet50face

import gdown

url = 'https://drive.google.com/uc?id=1oHJxVZCcVwp1dgcwDIZL4h97uInxOGWO'
output = 'resnet50face.h5'
gdown.download(url, output, quiet=False)


from tensorflow.keras.models import load_model

vggface_model = load_model("resnet50face.h5")

Downloading...
From (original): https://drive.google.com/uc?id=1oHJxVZCcVwp1dgcwDIZL4h97uInxOGWO
From (redirected): https://drive.google.com/uc?id=1oHJxVZCcVwp1dgcwDIZL4h97uInxOGWO&confirm=t&uuid=76373e65-f53a-4b54-bb7c-3dc796515afc
To: /content/resnet50face.h5
100%|██████████| 166M/166M [00:01<00:00, 95.5MB/s]


In [4]:
# Подключаем гугл диск

from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Прописываем путь к файлу с архивом данных для обучения и распаковываем его

import zipfile
import os

zip_file = '/content/drive/My Drive/Skillbox/train.zip'
z = zipfile.ZipFile(zip_file, 'r')
z.extractall()

In [6]:
# Прописываем путь к директории

folder_id = '/content/drive/My Drive/Skillbox'

In [7]:
# Загрузка учебного датасета

df = pd.read_csv("/content/drive/My Drive/Skillbox/train.csv")

In [8]:
# Функция для размечивания датасета на обучаемую и валидационную часть

from sklearn.model_selection import train_test_split

list_em = df['emotion'].unique()

def set_train_flag(df, emotion):
    indices = df[df['emotion'] == emotion].index
    train_indices = train_test_split(indices, test_size=0.3, random_state=42, shuffle=True)[0]
    df.loc[train_indices, 'train'] = 1
    return df

In [9]:
# Применяем функцию для каждой эмоции в списке

for em in list_em:
    df = set_train_flag(df.copy(), em)

# Разделяем датасет по метке
df_train = df[df["train"] == 1].reset_index()
df_val = df[df["train"] != 1].reset_index()

In [10]:
# Создаём генератор image_gen типа ImageDataGenerator

from PIL import Image
from keras import backend as K
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator


def preprocess_input_facenet(x, data_format=None, version=2):
    x_temp = np.copy(x)
    x_temp = image.img_to_array(x_temp)
    x_temp = np.expand_dims(x_temp, axis=0)
    if data_format is None:
        data_format = K.image_data_format()
    assert data_format in {'channels_last', 'channels_first'}


    if version == 1:
        if data_format == 'channels_first':
            x_temp = x_temp[:, ::-1, ...]
            x_temp[:, 0, :, :] -= 93.5940
            x_temp[:, 1, :, :] -= 104.7624
            x_temp[:, 2, :, :] -= 129.1863
        else:
            x_temp = x_temp[..., ::-1]
            x_temp[..., 0] -= 93.5940
            x_temp[..., 1] -= 104.7624
            x_temp[..., 2] -= 129.1863


    elif version == 2:
        if data_format == 'channels_first':
            x_temp = x_temp[:, ::-1, ...]
            x_temp[:, 0, :, :] -= 91.4953
            x_temp[:, 1, :, :] -= 103.8827
            x_temp[:, 2, :, :] -= 131.0912
        else:
            x_temp = x_temp[..., ::-1]
            x_temp[..., 0] -= 91.4953
            x_temp[..., 1] -= 103.8827
            x_temp[..., 2] -= 131.0912
    else:
        raise NotImplementedError


    return x_temp


image_gen = ImageDataGenerator(preprocessing_function=preprocess_input_facenet,
                               rotation_range=10,          # Поворот на случайный угол до 10 градусов
                               zoom_range=0.1,            # Увеличение размера до 10 %
                               width_shift_range=0.1,      # Сдвих влево / вправо до 10 процентов
                               height_shift_range=0.1,     # Сдвиг вверх / вниз до 10 процентов
)

In [11]:
# Создаём учебный и валидационный генератор

train_generator = image_gen.flow_from_dataframe(
    dataframe=df_train,
    directory=".",
    x_col="image_path",
    y_col="emotion",
    target_size=(224, 224),
    batch_size=64,
    shuffle=True,
    class_mode="categorical"
)

val_generator = image_gen.flow_from_dataframe(
    dataframe=df_val,
    directory=".",
    x_col="image_path",
    y_col="emotion",
    target_size=(224, 224),
    batch_size=64,
    shuffle=False,
    class_mode="categorical"
)

Found 35028 validated image filenames belonging to 9 classes.
Found 15018 validated image filenames belonging to 9 classes.


In [12]:
#  Чекпоинт для сохранения ВСЕЙ модели (архитектура + веса + оптимизатор)

checkpoint_callback_full = tf.keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(folder_id, "model_checkpoint_{epoch:02d}.hdf5"),
    monitor="categorical_accuracy",
    save_best_only=True,
    mode='max'
)

#  Чекпоинт для сохранения ТОЛЬКО весов

checkpoint_callback_weights = tf.keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(folder_id, "weights_only_{epoch:02d}.hdf5"),
    save_weights_only=True,
    monitor="val_categorical_accuracy",
    save_best_only=True,
    mode='max'
)

# Чекпоинт для уменьшения скорости обучения если контрольная метрика перестала улучшаться

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                                factor=0.1,
                                                patience=3,
                                                mode='auto',
                                                min_delta=0.01,
                                                min_lr=0.00001
)

In [13]:
# Отрезаем последний слой класификатора

base_model = tf.keras.Model([vggface_model.input], vggface_model.get_layer("flatten_1").output)

# Замораживаем всю базовую модель

base_model.trainable = False

In [14]:
# Добавляем к базовой модели класификатор


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.Dense(9, activation='softmax')
])


In [15]:
# Компилируем модель

from tensorflow.keras.optimizers import Adam

model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.01),
              loss='categorical_crossentropy',
              metrics=[tf.keras.metrics.CategoricalAccuracy()])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (Functional)          (None, 2048)              23561152  
                                                                 
 dense (Dense)               (None, 9)                 18441     
                                                                 
Total params: 23579593 (89.95 MB)
Trainable params: 18441 (72.04 KB)
Non-trainable params: 23561152 (89.88 MB)
_________________________________________________________________


In [16]:
# Загрузка всей модели из чекпоинта c гугл диска

model = tf.keras.models.load_model(os.path.join(folder_id, "model_checkpoint_06.hdf5"))

In [31]:
# Загрузка весов из чекпоинта c гугл диска

model.load_weights(os.path.join(folder_id, "weights_only_06.hdf5"))

In [27]:
# Обучаем модель

EPOCHS = 5
model.fit(train_generator, steps_per_epoch=25,
                              epochs=EPOCHS,
                              validation_data=val_generator,
                              callbacks=[checkpoint_callback_full, checkpoint_callback_weights, reduce_lr])

Epoch 1/10
25/25 [==============================] - ETA: 0s - loss: 1.3407 - categorical_accuracy: 0.5150

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


25/25 [==============================] - 304s 13s/step - loss: 1.3407 - categorical_accuracy: 0.5150 - val_loss: 1.5560 - val_categorical_accuracy: 0.4428 - lr: 0.0010
Epoch 2/10
25/25 [==============================] - 295s 12s/step - loss: 1.3883 - categorical_accuracy: 0.4831 - val_loss: 1.5484 - val_categorical_accuracy: 0.4467 - lr: 0.0010
Epoch 3/10
25/25 [==============================] - 273s 11s/step - loss: 1.3593 - categorical_accuracy: 0.5031 - val_loss: 1.5591 - val_categorical_accuracy: 0.4445 - lr: 0.0010
Epoch 4/10
25/25 [==============================] - 286s 12s/step - loss: 1.3215 - categorical_accuracy: 0.5213 - val_loss: 1.6650 - val_categorical_accuracy: 0.4086 - lr: 0.0010
Epoch 5/10
25/25 [==============================] - 281s 12s/step - loss: 1.3514 - categorical_accuracy: 0.5100 - val_loss: 1.4911 - val_categorical_accuracy: 0.4586 - lr: 1.0000e-04
Epoch 6/10
25/25 [==============================] - 292s 12s/step - loss: 1.2854 - categorical_accuracy: 0.5375 

In [18]:
# Прописываем путь к файлу с архивом тестовых файлов

zip_file = '/content/drive/My Drive/Skillbox/test_kaggle.zip'

# Распаковываем архив

z_test = zipfile.ZipFile(zip_file, 'r')
z_test.extractall()

In [19]:
# Загрузка тестового датасета

test_df = pd.read_csv("/content/drive/My Drive/Skillbox/sample_submission.csv")

In [20]:
#

text = './test_kaggle/'
test_df['image_path'] =  text + test_df['image_path']

In [23]:
# Создаём тестовый генератор

test_generator = image_gen.flow_from_dataframe(
    dataframe=test_df,
    directory=".",
    x_col="image_path",
    y_col="emotion",
    target_size=(224, 224),
    batch_size=64,
    class_mode="categorical",
    shuffle=False
)

Found 5000 validated image filenames belonging to 1 classes.


In [28]:
#  Предсказываем классы тестовых изображений

y_pred_proba = model.predict(test_generator)
y_pred = y_pred_proba.argmax(axis=1)

79/79 [==============================] - 82s 1s/step


In [29]:
# Создаём словарь эмоций и их числовой номерации

em_dick = {}

for i in range(0,9):
  em_dick[i] = list_em[i]

# Заменяем изначальные данные предсказанными классами

test_df = pd.read_csv("/content/drive/My Drive/Skillbox/sample_submission.csv")
test_df['emotion'] = y_pred
test_df['emotion'] = test_df['emotion'].replace(em_dick)

In [30]:
# Сохраняем csv файл

test_df.to_csv('result.csv', index=False)